In [44]:
%matplotlib inline
import pickle
import pandas as pd
from IPython.display import display

# Loading 2018 train data

In [45]:
with open('../data/processed/data.pkl', 'rb') as pf:
    objs = pickle.load(pf)

grammar_dic = objs[0]
df_18_A_train = objs[3]    # using RecResult
df_18_B_train = objs[4]
df_18_C_train = objs[5]

print('A shape {}'.format(df_18_A_train.shape))
print('B shape {}'.format(df_18_B_train.shape))
print('C shape {}'.format(df_18_C_train.shape))

A shape (5526, 8)
B shape (873, 8)
C shape (299, 8)


In [46]:
df_18_train = pd.concat([df_18_A_train, df_18_B_train, df_18_C_train])

In [47]:
recode = lambda x: 1 if x=='correct' else 0
df_18_train['language'] = df_18_train['language'].apply(recode)
df_18_train['meaning'] = df_18_train['meaning'].apply(recode)

# load various types of features
# grammar error count
dfA = pd.read_csv('../data/processed/df18_A_train_grmerror.csv')
dfB = pd.read_csv('../data/processed/df18_B_train_grmerror.csv')
dfC = pd.read_csv('../data/processed/df18_C_train_grmerror.csv')
train_grmerr = pd.concat([dfA, dfB, dfC])

# print(df_17_test_vecsim.shape)
print(train_grmerr.shape)

(6410, 12)


## Form a DF for modeling
- df_18_X, only using Id, language, and meaning cols
- using Id to merge other features DFs


- 1/24/2018 Huy's updated features (including edit pattern)

In [48]:
dfA = pd.read_csv('../data/processed/Huy/scst2_training_data_A_text_features.csv', sep='\t')
dfB = pd.read_csv('../data/processed/Huy/scst2_training_data_B_text_features.csv', sep='\t')
dfC = pd.read_csv('../data/processed/Huy/scst2_training_data_C_text_features.csv', sep='\t')

train_huy = pd.concat([dfA, dfB, dfC])
train_huy.rename(columns={'ID' : 'Id'}, inplace=True)

train_huy = train_huy.drop(['CLASS'], axis=1)
print(train_huy.shape)
print(train_huy.columns)

(6671, 195)
Index(['Id', 'ppl-ref', 'ppl-ref_pos', 'ppl-ref_prod', 'ppl-ref_dep',
       'ppl-prompt', 'ppl-prompt_pos', 'ppl-correct', 'ppl-correct_pos',
       'ppl-correct_prod',
       ...
       'length_unknown-ratio', 'length_sounds', 'length_sounds-ratio',
       'prompt_missing', 'prompt_missing-pct', 'prompt_DT', 'prompt_IN',
       'prompt_MD', 'prompt_NN', 'prompt_VB'],
      dtype='object', length=195)


In [49]:
def gen_ml_df(df_main, df_grmerr, df_huy):
    df_ml = df_main[['Id', 'language', 'meaning']]
    df_ml = pd.merge(df_ml, df_grmerr, on='Id')
    df_ml = pd.merge(df_ml, df_huy, on='Id')
    return df_ml

## Process 2018 train DF

In [50]:
train_ml = gen_ml_df(df_18_train, train_grmerr, train_huy)
print('tain_ml shape: {}'.format(train_ml.shape))
display(train_ml.describe())
print('#NA: {}'.format(train_ml.isnull().sum().sum()))

tain_ml shape: (6383, 208)


,Id,language,meaning,ofto_error_count,Spell,Article_a_an_error_count,sva_error_count,RuleRepeatedLinkWords_error_count,Verbform_error_count,noun_error_count,...,length_unknown-ratio,length_sounds,length_sounds-ratio,prompt_missing,prompt_missing-pct,prompt_DT,prompt_IN,prompt_MD,prompt_NN,prompt_VB
count,6383.000000,6383.000000,6383.000000,6383.000000,6383.0,6383.000000,6383.000000,6383.0,6383.000000,6383.000000,...,6383.0,6383.000000,6383.000000,6383.000000,6383.000000,6383.000000,6383.000000,6383.000000,6383.000000,6383.000000
mean,3314.567915,0.691838,0.858687,0.000313,0.0,0.001253,0.016607,0.0,0.000627,0.024440,...,0.0,5.230299,0.952217,1.331662,0.296593,0.153063,0.078490,0.073476,0.303149,0.252389
std,1924.820882,0.461770,0.348371,0.017700,0.0,0.035383,0.127802,0.0,0.025027,0.156439,...,0.0,2.291150,0.213324,1.578256,0.338235,0.360076,0.268962,0.260937,0.459655,0.434417
min,4.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1662.500000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3282.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,5.000000,1.000000,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4981.500000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,6.000000,1.000000,2.000000,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000
max,6698.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,1.000000,2.000000,...,0.0,89.000000,1.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#NA: 0


## Correlation analysis
Only show |R| more than $CORR_{CUT}$

In [51]:
tb_corr = train_ml.corr()
CORR_CUT = 0.1

tb_l = tb_corr['language'].drop(['language', 'meaning'])
display(tb_l[tb_l > CORR_CUT])
display(tb_l[tb_l < -1 * CORR_CUT])
col_l = list(tb_l[tb_l > CORR_CUT].index) + list(tb_l[tb_l < -1 * CORR_CUT].index)
display(col_l)
len(col_l)

ppl-ref              0.299982
ppl-ref_pos          0.179849
ppl-ref_prod         0.107081
ppl-ref_dep          0.142977
ppl-prompt_pos       0.129655
ppl-correct          0.324922
ppl-correct_pos      0.175772
ppl-correct_prod     0.134920
maxsim_15_skip       0.348486
maxsim_30_skip       0.353502
maxsim_50_skip       0.348165
maxsim_15_cbw        0.356029
maxsim_30_cbw        0.362005
maxsim_50_cbw        0.355592
lda_sim-max          0.339177
lda_sim-min          0.190316
lda_sim-avg          0.284269
ngram_match          0.199233
ngram_match-lem      0.188534
parse_score-ratio    0.115689
length_ratio         0.108930
Name: language, dtype: float64

noun_error_count      -0.112632
Article_error_count   -0.191626
edit_distance         -0.428848
edit_pattern-JJ-NN    -0.102209
edit_pattern-NN-VB    -0.124359
edit_pattern-NNS-NN   -0.114270
edit_pattern-OO-CD    -0.108726
edit_pattern-OO-DT    -0.132904
edit_pattern-OO-FW    -0.101921
edit_pattern-OO-IN    -0.144808
edit_pattern-OO-JJ    -0.135984
edit_pattern-OO-NN    -0.201263
edit_pattern-OO-TO    -0.183642
edit_pattern-OO-VB    -0.135089
edit_pattern-OO-VBP   -0.135041
edit_pattern-VB-VBP   -0.102068
ngram_unseen-1        -0.401850
ngram_unseen-2        -0.434220
length_under-min      -0.235041
length_above-max      -0.118725
prompt_missing        -0.414655
prompt_missing-pct    -0.340973
prompt_DT             -0.135631
prompt_IN             -0.232908
prompt_NN             -0.316479
prompt_VB             -0.243354
Name: language, dtype: float64

['ppl-ref',
 'ppl-ref_pos',
 'ppl-ref_prod',
 'ppl-ref_dep',
 'ppl-prompt_pos',
 'ppl-correct',
 'ppl-correct_pos',
 'ppl-correct_prod',
 'maxsim_15_skip',
 'maxsim_30_skip',
 'maxsim_50_skip',
 'maxsim_15_cbw',
 'maxsim_30_cbw',
 'maxsim_50_cbw',
 'lda_sim-max',
 'lda_sim-min',
 'lda_sim-avg',
 'ngram_match',
 'ngram_match-lem',
 'parse_score-ratio',
 'length_ratio',
 'noun_error_count',
 'Article_error_count',
 'edit_distance',
 'edit_pattern-JJ-NN',
 'edit_pattern-NN-VB',
 'edit_pattern-NNS-NN',
 'edit_pattern-OO-CD',
 'edit_pattern-OO-DT',
 'edit_pattern-OO-FW',
 'edit_pattern-OO-IN',
 'edit_pattern-OO-JJ',
 'edit_pattern-OO-NN',
 'edit_pattern-OO-TO',
 'edit_pattern-OO-VB',
 'edit_pattern-OO-VBP',
 'edit_pattern-VB-VBP',
 'ngram_unseen-1',
 'ngram_unseen-2',
 'length_under-min',
 'length_above-max',
 'prompt_missing',
 'prompt_missing-pct',
 'prompt_DT',
 'prompt_IN',
 'prompt_NN',
 'prompt_VB']

47

In [52]:
# convert pandas DF to numpy array

def get_langauge_X(df):
    X = df.loc[:, col_l].values
    return X

def get_langauge_y(df):
    return df['language'].values

def get_meaning_y(df):
    return df['meaning'].values

def get_both_y(df):
    return df.loc[:,['meaning', 'language']].values

## Split the entire train to train and dev

In [53]:
from sklearn.model_selection import train_test_split

X = get_langauge_X(train_ml)
y = get_both_y(train_ml)

lang_train_X, lang_test_X, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

# from two labels to one label
lang_train_y = y_train[:,1]
meaning_train_y = y_train[:,0]
lang_test_y = y_test[:,1]
meaning_test_y = y_test[:,0]

print(lang_train_X.shape)
print(lang_train_y.shape)

print(lang_test_X.shape)
print(lang_test_y.shape)

print(meaning_train_y.shape)
print(meaning_test_y.shape)

(5106, 47)
(5106,)
(1277, 47)
(1277,)
(5106,)
(1277,)


## z-norm all features

In [54]:
from sklearn.preprocessing.data import StandardScaler, MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1)) # for SVM
#scaler = StandardScaler()

In [55]:
scaler.fit(lang_train_X)
lang_train_X = scaler.transform(lang_train_X)
lang_test_X = scaler.transform(lang_test_X)

print(lang_train_X.shape)
print(lang_train_y.shape)
print(lang_test_X.shape)
print(lang_test_y.shape)

(5106, 47)
(5106,)
(1277, 47)
(1277,)


Pickle all year18 numpy arraies

In [56]:
with open('../data/processed/numpy/year18.pkl', 'wb') as pf:
    pickle.dump([lang_train_X,
                 lang_train_y,
                 lang_test_X,
                 lang_test_y,
                 meaning_train_y,
                 meaning_test_y], pf)
